In [2]:
from typing import List
from datetime import datetime, timedelta
import secrets

from sqlalchemy.orm import Session

from sqlalchemy import asc, or_, and_, func, not_
from sqlalchemy.sql import text

from sql_app import crud, models, schemas
from sql_app.database import SessionLocal, engine

models.Base.metadata.create_all(bind=engine)

db = SessionLocal()

date_from = datetime.now()
date_to = date_from+timedelta(days=10)
try:
    available_costumes = db.query(models.CostumeItem).join(models.Reservation,isouter=True).\
        filter(or_(or_(date_to < models.Reservation.pick_up_date, date_from > models.Reservation.return_date),\
               models.CostumeItem.reservation_id == None))
    all_costumes = db.query(models.CostumeItem)
    items = db.query(models.CostumeItem).filter_by(reservation_id=None).order_by(models.CostumeItem.id.asc())
    
finally:
    db.close()


In [13]:
[c.__dict__ for c in items.all()]
print(items)

SELECT "Costume_items".costume_id AS "Costume_items_costume_id", "Costume_items".model_id AS "Costume_items_model_id", "Costume_items".location_id AS "Costume_items_location_id", "Costume_items".rental_id AS "Costume_items_rental_id", "Costume_items".reservation_id AS "Costume_items_reservation_id" 
FROM "Costume_items" 
WHERE "Costume_items".reservation_id IS NULL ORDER BY "Costume_items".costume_id ASC


In [11]:
#costumes = schemas.CostumeItem(**available_costumes.__dict__)
for c in available_costumes.all():
    print(c.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1ca0>, 'reservation_id': None, 'location_id': 2, 'id': 4, 'rental_id': None, 'model_id': 4}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f02397443d0>, 'reservation_id': None, 'location_id': 2, 'id': 5, 'rental_id': None, 'model_id': 4}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0239744250>, 'reservation_id': None, 'location_id': 1, 'id': 7, 'rental_id': None, 'model_id': 7}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0239744880>, 'reservation_id': 1, 'location_id': 1, 'id': 2, 'rental_id': None, 'model_id': 2}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f02397440d0>, 'reservation_id': 1, 'location_id': 1, 'id': 6, 'rental_id': None, 'model_id': 6}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f02208d35b0>, 'reservation_id': 2, 'location_id': 1, 'id': 1, 'rental_id': None

In [4]:
for c in all_costumes.all():
    print(c.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1a00>, 'reservation_id': None, 'location_id': 2, 'id': 4, 'rental_id': None, 'model_id': 4}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1880>, 'reservation_id': None, 'location_id': 2, 'id': 5, 'rental_id': None, 'model_id': 4}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1970>, 'reservation_id': None, 'location_id': 1, 'id': 6, 'rental_id': None, 'model_id': 6}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1670>, 'reservation_id': None, 'location_id': 2, 'id': 8, 'rental_id': None, 'model_id': 3}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1700>, 'reservation_id': 11, 'location_id': 1, 'id': 2, 'rental_id': None, 'model_id': 2}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1760>, 'reservation_id': 11, 'location_id': 1, 'id': 3, 'rental_id':

In [5]:
len(available_costumes.all())

8

In [20]:
db = SessionLocal()

date_from = datetime.now()
date_to = date_from+timedelta(days=10)
try:
    available_costumes = db.query(models.CostumeItem).\
        filter(models.CostumeItem.rental_id == None)
finally:
    db.close()

In [22]:
str(available_costumes)

'SELECT "Costume_items".costume_id AS "Costume_items_costume_id", "Costume_items".model_id AS "Costume_items_model_id", "Costume_items".location_id AS "Costume_items_location_id", "Costume_items".rental_id AS "Costume_items_rental_id", "Costume_items".reservation_id AS "Costume_items_reservation_id" \nFROM "Costume_items" \nWHERE "Costume_items".rental_id IS NULL'

In [21]:
available_costumes.all()

In [30]:
costumes= [
    {
      "model_id": 4,
      "quantity": 2
    },
    {
      "model_id": 7,
      "quantity": 1
    },
    {
      "model_id": 1,
      "quantity": 7
    }
  ]

count = {4:2, 7:1}

In [33]:
for id_, quan in zip(count, count.values()):
    print(f'id: {id_}, quantity:{quan}')

id: 4, quantity:2
id: 7, quantity:1


[{'model_id': 4, 'quantity': 2},
 {'model_id': 7, 'quantity': 1},
 {'model_id': 1, 'quantity': 7}]

# delete all reservations and rentals

In [5]:
db = SessionLocal()

date_from = datetime.now()
date_to = date_from+timedelta(days=10)
try:
    available_costumes = db.query(models.CostumeItem).join(models.Reservation).\
        filter(or_(date_to < models.Reservation.pick_up_date, date_from > models.Reservation.return_date))
    items = db.query(models.CostumeItem).all()
    #db.query(models.Reservation).delete()
    for item in items:
        item.reservation_id = None
        item.rental_id = None
    db.commit() #uncomment it when u really want to deleta all reservations
    
    db.query(models.CostumeRental).delete()
    db.query(models.Reservation).delete()
    
    db.commit()
finally:
    db.close()